In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
cd /content/drive/My Drive/echo/

/content/drive/My Drive/echo


In [ ]:
!pip install simpleITK

     |████████████████████████████████| 42.5MB 108kB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from glob import glob
import numpy as np
import os
import skimage.io as io
from PIL import Image
from prefetch_generator import background
from keras.preprocessing.image import ImageDataGenerator
import random

NUM_PREFETCH = 10
RANDOM_SEED = 123


class DataLoaderCamus:
    def __init__(self, dataset_path, input_name, target_name, condition_name,
                 img_res, target_rescale, input_rescale, condition_rescale, train_ratio, valid_ratio,
                 labels, augment):
        self.dataset_path = dataset_path
        self.img_res = tuple(img_res)
        self.target_rescale = target_rescale
        self.input_rescale = input_rescale
        self.condition_rescale = condition_rescale
        self.input_name = input_name
        self.target_name = target_name
        self.condition_name = condition_name
        self.augment = augment

        patients = sorted(glob(os.path.join(self.dataset_path, 'training', '*')))
        random.Random(RANDOM_SEED).shuffle(patients)
        num = len(patients)
        num_train = int(num * train_ratio)
        num_valid = int(num_train * valid_ratio)

        self.valid_patients = patients[:num_valid]
        self.train_patients = patients[num_valid:num_train]
        self.test_patients = patients[num_train:]
        if train_ratio == 1.0:
            self.test_patients = glob(os.path.join(self.dataset_path, 'testing', '*'))
        print('#train:', len(self.train_patients))
        print('#valid:', len(self.valid_patients))
        print('#test:', len(self.test_patients))

        all_labels = {0, 1, 2, 3}
        self.not_labels = all_labels - set(labels)

        data_gen_args = dict(rotation_range=augment['AUG_ROTATION_RANGE_DEGREES'],
                             width_shift_range=augment['AUG_WIDTH_SHIFT_RANGE_RATIO'],
                             height_shift_range=augment['AUG_HEIGHT_SHIFT_RANGE_RATIO'],
                             shear_range=augment['AUG_SHEAR_RANGE_ANGLE'],
                             zoom_range=augment['AUG_ZOOM_RANGE_RATIO'],
                             fill_mode='constant',
                             cval=0.,
                             data_format='channels_last')
        self.datagen = ImageDataGenerator(**data_gen_args)

    def read_mhd(self, img_path, is_gt):
        if not os.path.exists(img_path):
            return np.zeros(self.img_res + (1,))
        img = io.imread(img_path, plugin='simpleitk').squeeze()
        img = np.array(Image.fromarray(img).resize(self.img_res))
        img = np.expand_dims(img, axis=2)

        if is_gt:
            for not_l in self.not_labels:
                img[img == not_l] = 0
        return img

    def _get_paths(self, stage):
        if stage == 'train':
            return self.train_patients
        elif stage == 'valid':
            return self.valid_patients
        elif stage == 'test':
            return self.test_patients

    @background(max_prefetch=NUM_PREFETCH)
    def get_random_batch(self, batch_size=1, stage='train'):
        paths = self._get_paths(stage)

        num = len(paths)
        num_batches = num // batch_size

        for i in range(num_batches):
            batch_paths = np.random.choice(paths, size=batch_size)
            target_imgs, condition_imgs, input_imgs, weight_imgs = self._get_batch(batch_paths, stage)
            target_imgs = target_imgs * self.target_rescale
            input_imgs = input_imgs * self.input_rescale
            condition_imgs = condition_imgs * self.condition_rescale

            yield target_imgs, condition_imgs, input_imgs, weight_imgs

    def get_iterative_batch(self, batch_size=1, stage='test'):
        paths = self._get_paths(stage)

        num = len(paths)
        num_batches = num // batch_size

        start_idx = 0
        for i in range(num_batches):
            batch_paths = paths[start_idx:start_idx + batch_size]
            target_imgs, condition_imgs, input_imgs, weight_imgs = self._get_batch(batch_paths, stage)
            target_imgs = target_imgs * self.target_rescale
            input_imgs = input_imgs * self.input_rescale
            condition_imgs = condition_imgs * self.condition_rescale
            start_idx += batch_size

            yield target_imgs, condition_imgs, input_imgs, weight_imgs

    def _get_batch(self, paths_batch, stage):
        target_imgs = []
        input_imgs = []
        condition_imgs = []
        weight_maps = []

        for path in paths_batch:
            transform = self.datagen.get_random_transform(img_shape=self.img_res)
            head, patient_id = os.path.split(path)
            target_path = os.path.join(path, '{}_{}.mhd'.format(patient_id, self.target_name))
            condition_path = os.path.join(path, '{}_{}.mhd'.format(patient_id, self.condition_name))
            input_path = os.path.join(path, '{}_{}.mhd'.format(patient_id, self.input_name))

            input_img = self.read_mhd(input_path, '_gt' in self.input_name)
            if self.augment['AUG_INPUT']:
                input_img = self.datagen.apply_transform(input_img, transform)
            input_imgs.append(input_img)

            target_img = self.read_mhd(target_path, '_gt' in self.target_name)
            condition_img = self.read_mhd(condition_path, 1)

            if self.augment['AUG_TARGET']:
                if not self.augment['AUG_SAME_FOR_BOTH']:
                    transform = self.datagen.get_random_transform(img_shape=self.img_res)
                target_img = self.datagen.apply_transform(target_img, transform)
                condition_img = self.datagen.apply_transform(condition_img, transform)
            target_imgs.append(target_img)
            condition_imgs.append(condition_img)

            weight_map_condition = self.get_weight_map(condition_img)
            weight_maps.append(weight_map_condition)

        return np.array(target_imgs), np.array(condition_imgs), np.array(input_imgs), np.array(weight_maps)

    def get_weight_map(self, mask):
        # let the y axis have higher variance
        gauss_var = [[self.img_res[0] * 60, 0], [0, self.img_res[1] * 30]]
        x, y = mask[:, :, 0].nonzero()
        center = [x.mean(), y.mean()]

        from scipy.stats import multivariate_normal
        gauss = multivariate_normal.pdf(np.mgrid[
                                        0:self.img_res[1],
                                        0:self.img_res[0]].reshape(2, -1).transpose(),
                                        mean=center,
                                        cov=gauss_var)
        gauss /= gauss.max()
        gauss = gauss.reshape((self.img_res[1], self.img_res[0], 1))

        # set the gauss value of the main target part to 1
        gauss[mask > 0] = 1

        return gauss


Using TensorFlow backend.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import keras.backend as K

plt.switch_backend('agg')


def gen_fig(inputs, generated, targets):
    r, c = 3, 3
    titles = ['Condition', 'Generated', 'Original']
    all_imgs = np.concatenate([inputs, generated, targets])

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(all_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].set_title(titles[i], fontdict={'fontsize': 8})
            axs[i, j].axis('off')
            cnt += 1
    return fig


def set_backend():
    from keras.optimizers import tf
    cf = tf.ConfigProto()
    cf.gpu_options.allow_growth = True
    sess = tf.Session(config=cf)
    K.set_session(sess)


def weighted_mae(weight_map):
    def mae(y_true, y_pred):
        return K.mean(K.abs(y_true - y_pred) * weight_map)
    return mae


In [ ]:
"""
The implementations of models are based on the pix2pix implementation of the Keras-GAN library:
https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/pix2pix.py
"""

import tensorflow as tf
from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dropout, Concatenate
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2DTranspose, Conv2D
from keras.models import Model


'''class UNetGenerator:
    def __init__(self, img_shape, filters, channels, output_activation, skip_connections):
        self.img_shape = img_shape
        self.filters = filters
        self.channels = channels
        self.output_activation = output_activation
        self.skip_connection = skip_connections

    def build(self):
        def conv2d(layer_input, filters, f_size=4, bn=True):
            d = Conv2D(filters, kernel_size=f_size,
                       strides=2, padding='same')(layer_input)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            d = LeakyReLU(alpha=0.2)(d)

            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            u = Conv2DTranspose(filters, kernel_size=f_size, strides=(2, 2),
                                padding='same', activation='linear')(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1,
                       padding='same', activation='relu')(u)

            u = BatchNormalization(momentum=0.8)(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            if self.skip_connection:
                u = Concatenate()([u, skip_input])

            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling: 7 x stride of 2 --> x1/128 downsampling
        d1 = conv2d(d0, self.filters, bn=False)
        d2 = conv2d(d1, self.filters * 2)
        d3 = conv2d(d2, self.filters * 4)
        d4 = conv2d(d3, self.filters * 8)
        d5 = conv2d(d4, self.filters * 8)
        d6 = conv2d(d5, self.filters * 8)
        d7 = conv2d(d6, self.filters * 8)

        # Upsampling: 6 x stride of 2 --> x64 upsampling
        u1 = deconv2d(d7, d6, self.filters * 8)
        u2 = deconv2d(u1, d5, self.filters * 8)
        u3 = deconv2d(u2, d4, self.filters * 8)
        u4 = deconv2d(u3, d3, self.filters * 4)
        u5 = deconv2d(u4, d2, self.filters * 2)
        u6 = deconv2d(u5, d1, self.filters)
        u7 = Conv2DTranspose(self.channels, kernel_size=4, strides=(2, 2),
                             padding='same', activation='linear')(u6)

        # added conv layers after the deconvs to avoid the pixelated outputs
        output_img = Conv2D(self.channels, kernel_size=4,
                            strides=1, padding='same',
                            activation=self.output_activation)(u7)

        return Model(d0, output_img)'''


class RESUnet:
    def __init__(self, img_shape, filters, channels, output_activation, skip_connections):
        self.img_shape = img_shape
        self.filters = filters
        self.channels = channels
        self.output_activation = output_activation
        self.skip_connection = skip_connections    

    def build(self):
        def bn_act(x, act=True):
            x = tf.keras.layers.BatchNormalization()(x)
            if act == True:
                x = tf.keras.layers.Activation('relu')(x)
            return x

        #convolutional layer, always uses the batch normalization layer
        def conv_block(x, filters, kernel_size=3, padding='same', strides=1):
            conv = bn_act(x)
            conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
            return conv

        def stem(x, filters, kernel_size=3, padding='same', strides=1):
            conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
            conv = conv_block(conv, filters, kernel_size, padding, strides)
            shortcut = Conv2D(filters, kernel_size=1, padding=padding, strides=strides)(x)
            shortcut = bn_act(shortcut, act=False)
            output = Add()([conv, shortcut])
            return output

        #Residual block/unit from Resnet with binary normalization as skip connection
        def residual_block(x, filters, kernel_size=3, padding='same', strides=1):
            res = conv_block(x, filters, k_size, padding, strides)
            res = conv_block(res, filters, k_size, padding, 1)
            shortcut = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
            shortcut = bn_act(shortcut, act=False)
            output = Add()([shortcut, res])
            return output

        #upsampling block (Decoder path)
        def upsample_concat_block(x, xskip):
            u = UpSampling2D((2,2))(x)
            c = Concatenate()([u, xskip])
            return c

        #ResUNet model
        def ResUNet(img_h, img_w):
            f = [16, 32, 64, 128, 256]
            inputs = Input((img_h, img_w, 1))
            
            ## Encoder
            e0 = inputs
            e1 = stem(e0, f[0])
            e2 = residual_block(e1, f[1], strides=2)
            e3 = residual_block(e2, f[2], strides=2)
            e4 = residual_block(e3, f[3], strides=2)
            e5 = residual_block(e4, f[4], strides=2)
            
            ## Bridge
            b0 = conv_block(e5, f[4], strides=1)
            b1 = conv_block(b0, f[4], strides=1)
            
            ## Decoder
            u1 = upsample_concat_block(b1, e4)
            d1 = residual_block(u1, f[4])
            
            u2 = upsample_concat_block(d1, e3)
            d2 = residual_block(u2, f[3])
            
            u3 = upsample_concat_block(d2, e2)
            d3 = residual_block(u3, f[2])
            
            u4 = upsample_concat_block(d3, e1)
            d4 = residual_block(u4, f[1])
            
            outputs = tf.keras.layers.Conv2D(4, (1, 1), padding="same", activation="sigmoid")(d4)
            model = tf.keras.models.Model(inputs, outputs)
            return model

    def dsc(y_true, y_pred):
        smooth = 1.
        y_true_f = Flatten()(y_true)
        y_pred_f = Flatten()(y_pred)
        intersection = reduce_sum(y_true_f * y_pred_f)
        score = (2. * intersection + smooth) / (reduce_sum(y_true_f) + reduce_sum(y_pred_f) + smooth)
        return score

    def dice_loss(y_true, y_pred):
        loss = 1 - dsc(y_true, y_pred)
        return loss

    def bce_dice_loss(y_true, y_pred):
        loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
        return loss

    



In [ ]:
import json
from absl import app
from absl import flags
import matplotlib.pyplot as plt

#from data_loader_camus import DataLoaderCamus
#from models import RESUnet;
#from utils import set_backend


flags.DEFINE_string('dataset_path', "/content/drive/My Drive/Neural Network Dataset/", 'Path of the dataset.')
flags.DEFINE_boolean('test', False, 'Test model and generate outputs on the test set')
flags.DEFINE_string('config', "configs/ventricle.json", 'Config file for training hyper-parameters.')
flags.DEFINE_boolean('use_wandb', False, 'Use wandb for logging')
flags.DEFINE_string('ckpt_load', None, 'Path to load the model')
flags.DEFINE_float('train_ratio', 0.95,
                   'Ratio of training data used for training and the rest used for testing. Set this value to 1.0 if '
                   'the data in the test folder are to be used for testing.')
flags.DEFINE_float('valid_ratio', 0.02, 'Ratio of training data used for validation')

#FLAGS = flags.FLAGS


plt.switch_backend('agg')


def main(argv):
    # Load configs from file
    config = json.load(open(FLAGS.config))
    set_backend()

    # Set name
    name = '{}_{}_'.format(config['INPUT_NAME'], config['TARGET_NAME'])
    for l in config['LABELS']:
        name += str(l)
    config['NAME'] += '_' + name

    # Organize augmentation hyper-parameters from config
    augmentation = dict()
    for key, value in config.items():
        if 'AUG_' in key:
            augmentation[key] = value

    # Initialize data loader
    data_loaderED = DataLoaderCamus(
        dataset_path=FLAGS.dataset_path,
        input_name=config['INPUT_NAME'],
        target_name=config['TARGET_NAME'],
        condition_name=config['CONDITION_NAME'],
        img_res=config['IMAGE_RES'],
        target_rescale=config['TARGET_TRANS'],
        input_rescale=config['INPUT_TRANS'],
        condition_rescale=config['CONDITION_TRANS'],
        labels=config['LABELS'],
        train_ratio=FLAGS.train_ratio,
        valid_ratio=FLAGS.valid_ratio,
        augment=augmentation
    )

    data_loaderES = DataLoaderCamus(
        dataset_path=FLAGS.dataset_path,
        input_name="4CH_ES_gt",
        target_name="4CH_ES",
        condition_name="4CH_ES_gt",
        img_res=config['IMAGE_RES'],
        target_rescale=config['TARGET_TRANS'],
        input_rescale=config['INPUT_TRANS'],
        condition_rescale=config['CONDITION_TRANS'],
        labels=config['LABELS'],
        train_ratio=FLAGS.train_ratio,
        valid_ratio=FLAGS.valid_ratio,
        augment=augmentation
    )





    # build model for ED
    #model = EjectionFraction(data_loader, config, FLAGS.use_wandb)
    modelED = RESUNet(self.img_shape, self.df, self.channels, self.output_activation, self.skipconnections_generator).build()
    adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)
    modelED.compile(optimizer=adam, loss=bce_dice_loss, metrics=[dsc])
    #modelED.summary() # print out the architecture of our network

    # build model for ES
    #model = EjectionFraction(data_loader, config, FLAGS.use_wandb)
    modelES = RESUNet(self.img_shape, self.df, self.channels, self.output_activation, self.skipconnections_generator).build()
    adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)
    modelES.compile(optimizer=adam, loss=bce_dice_loss, metrics=[dsc])
    #modelES.summary() # print out the architecture of our network


    # load a pre trained model here if you wish
    # model.load_weights('../input/resunet-e200-s256/ResUNet.h5')

    # running more epoch to see if we can get better results
    historyED = modelED.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=10, verbose=1)
    historyES = modelES.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=10, verbose=1)

    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.figure(figsize=(20,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['dsc'])
    plt.plot(history.history['val_dsc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')







    # load trained models if they exist
    if FLAGS.ckpt_load is not None:
        model.load_model(FLAGS.ckpt_load)

    if FLAGS.test:
        model.test()
    else:
        model.train()


if __name__ == '__main__':
    app.run(main)


DuplicateFlagError: ignored